Getting data from web about pincodes of the Bengaluru District

In [1]:
import pandas as pd
df=pd.read_html("https://www.mapsofindia.com/pincode/india/karnataka/bangalore/")

In [2]:
len(df)

1

In [3]:
df=df[0]

In [4]:
df.head()

,Pincode Details,Pincode Details.1,Pincode Details.2,Pincode Details.3
0,Location,Pincode,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore


In [5]:
df = df.drop([0],axis=0)

In [13]:
df=df.rename(columns={"Pincode Details":"Location","Pincode Details.1":"Pin Code","Pincode Details.2":"State","Pincode Details.3":"District"})

In [14]:
df

,Location,Pin Code,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore
5,Air Force hospital,560007,Karnataka,Bangalore
...,...,...,...,...
290,Yelahanka,560064,Karnataka,Bangalore
291,Yelahanka Satellite town,560064,Karnataka,Bangalore
292,Yemalur,560037,Karnataka,Bangalore
293,Yeshwanthpur Bazar,560022,Karnataka,Bangalore


### Getting the latitudes and longitudes

In [15]:
from geopy.geocoders import Nominatim

In [16]:
la=list()
lt=list()
poc=list()

In [17]:
df

,Location,Pin Code,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore
5,Air Force hospital,560007,Karnataka,Bangalore
...,...,...,...,...
290,Yelahanka,560064,Karnataka,Bangalore
291,Yelahanka Satellite town,560064,Karnataka,Bangalore
292,Yemalur,560037,Karnataka,Bangalore
293,Yeshwanthpur Bazar,560022,Karnataka,Bangalore


In [18]:
for index , rows in df.iterrows():
    address = rows["Location"] + "," + "Karnataka"
    geolocator = Nominatim(user_agent = "ben_exp")
    location = geolocator.geocode(address)
    try:
        longitude = location.longitude
        latitude = location.latitude
    except:
        longitude = "Not Found"
        latitude = "Not Found"
    poc.append(rows["Pin Code"])
    la.append(latitude)
    lt.append(longitude)
    
   

In [19]:
df.insert(4,"Pin",poc)
df.insert(5,"Latitude",la)
df.insert(6,"Longitude",lt)

In [20]:
df.head()

,Location,Pin Code,State,District,Pin,Latitude,Longitude
1,A F station yelahanka,560063,Karnataka,Bangalore,560063,Not Found,Not Found
2,Adugodi,560030,Karnataka,Bangalore,560030,12.9428,77.6104
3,Agara,560034,Karnataka,Bangalore,560034,13.0144,78.0489
4,Agram,560007,Karnataka,Bangalore,560007,Not Found,Not Found
5,Air Force hospital,560007,Karnataka,Bangalore,560007,Not Found,Not Found


In [21]:
df.to_csv(r"C:\Users\Ravi\Desktop\projects\bengaluru.csv")

In [26]:
df[df["Latitude"]=="Not Found"]


,Location,Pin Code,State,District,Pin,Latitude,Longitude
1,A F station yelahanka,560063,Karnataka,Bangalore,560063,Not Found,Not Found
4,Agram,560007,Karnataka,Bangalore,560007,Not Found,Not Found
5,Air Force hospital,560007,Karnataka,Bangalore,560007,Not Found,Not Found
9,Anekalbazar,562106,Karnataka,Bangalore,562106,Not Found,Not Found
17,Avani Sringeri mutt,560086,Karnataka,Bangalore,560086,Not Found,Not Found
...,...,...,...,...,...,...,...
280,Viswaneedam,560091,Karnataka,Bangalore,560091,Not Found,Not Found
281,Vittalnagar,560018,Karnataka,Bangalore,560018,Not Found,Not Found
283,Vyalikaval Extn,560003,Karnataka,Bangalore,560003,Not Found,Not Found
293,Yeshwanthpur Bazar,560022,Karnataka,Bangalore,560022,Not Found,Not Found


### Dropping the rows without latitude and longitude

In [31]:
df.drop(df[df["Latitude"]=="Not Found"].index,inplace=True)

In [39]:
df.reset_index(inplace=True)

In [40]:
df.drop("index",inplace=True,axis=1)

In [47]:
df["Pin Code"].unique().size

83

In [48]:
df.drop("Pin",axis=1,inplace=True)

In [52]:
address= "Bengaluru , Karnataka"
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude= location.longitude

In [55]:
import folium
map_ben= folium.Map(location=[Latitude,Longitude],zoom_start=10)
for lat,lan,loc,pin in zip(df["Latitude"],df["Longitude"],df["Location"],df["Pin Code"]):
    label="{},{}".format(loc,pin)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lan],
    radius=5,
    popup=label,
    color='black',
    fill_color="#3185cc",
    fill_opactiy=0.7,
    parse_html=False).add_to(map_ben)
map_ben
